Intro) Dans les jeux Pokemon, les pokémons sont regroupés dans ce que l'on appelle des "groupes d'oeufs". Ces groupes déterminent quels pokemons peuvent se reproduire entre eux et sont par conséquent similaire à la notion de "genre animal". Par exemple, le lion et le tigre font parti du même genre et peuve ainsi donner naissance à des hybrides.

Le but de ce projet est d'essayer de construire un modèle de prediction de l'appartenance ou non d'un pokémon aux différents groupes d'oeufs (CLASSIFICATION MULTILABEL)

Dans ce notebook, on lance un crawl via selenium pour recuperer les informations des pokemons disponibles sur wikipedia ainsi que les cris des pokemons sous format MP3. Ces informations seront ensuite utilisées pour constuire des séries temporelles en csv à partir des mp3 récupérés : Notebook 2

In [1]:
import re
import time
import json
import os
import warnings
import pandas as pd
import concurrent.futures
import random
import requests

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

1) Préparation des urls
- On note les différents urls des pages listant l'ensemble des pokemons

In [2]:
urls_generations = ["https://www.pokepedia.fr/Liste_des_Pok%C3%A9mon_de_la_premi%C3%A8re_g%C3%A9n%C3%A9ration",
                   "https://www.pokepedia.fr/Liste_des_Pok%C3%A9mon_de_la_deuxi%C3%A8me_g%C3%A9n%C3%A9ration",
                   "https://www.pokepedia.fr/Liste_des_Pok%C3%A9mon_de_la_troisi%C3%A8me_g%C3%A9n%C3%A9ration",
                   "https://www.pokepedia.fr/Liste_des_Pok%C3%A9mon_de_la_quatri%C3%A8me_g%C3%A9n%C3%A9ration",
                   "https://www.pokepedia.fr/Liste_des_Pok%C3%A9mon_de_la_cinqui%C3%A8me_g%C3%A9n%C3%A9ration",
                   "https://www.pokepedia.fr/Liste_des_Pok%C3%A9mon_de_la_sixi%C3%A8me_g%C3%A9n%C3%A9ration",
                   "https://www.pokepedia.fr/Liste_des_Pok%C3%A9mon_de_la_septi%C3%A8me_g%C3%A9n%C3%A9ration",
                   "https://www.pokepedia.fr/Liste_des_Pok%C3%A9mon_de_la_huiti%C3%A8me_g%C3%A9n%C3%A9ration",
                   "https://www.pokepedia.fr/Liste_des_Pok%C3%A9mon_de_la_neuvi%C3%A8me_g%C3%A9n%C3%A9ration"]

2) Création du chrome driver

In [3]:
#Setting up chrome deriver
chrome_options = Options()
path = r"DRIVER_PATH"
#chrome_options.add_argument("--headless")
chrome = webdriver.Chrome(executable_path = path, options = chrome_options)
chrome.maximize_window()


3) Récupération des urls français et allemands des pokemons. 
- On prend la version allemande également car sur les pages en français, les cris audios sont uniquement disponibles en format ogg.

In [4]:
#Looping on the urls to get the pokemon links
urls_fr= []
urls_de= []
for url in tqdm(urls_generations) :
    chrome.get(url)
    time.sleep(1)
    table = chrome.find_element(By.XPATH,'//*[@id="mw-content-text"]/div[1]/table[2]/tbody')
    rows = table.find_elements(By.TAG_NAME,"tr")
    for row in rows : 
        #Fr pour info
        url_fr = row.find_elements(By.TAG_NAME,"td")[2]
        url_fr= url_fr.find_element(By.TAG_NAME,"a").get_attribute("href")
        urls_fr.append(url_fr)
        
        #De pour audio en MP3
        url_de = row.find_elements(By.TAG_NAME,"td")[4]
        url_de= url_de.find_element(By.TAG_NAME,"a").get_attribute("href")
        urls_de.append(url_de)

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [01:12<00:00,  8.05s/it]


4) On récupère les informations nécessaires
- Nom du pokemon
- Numéro pokédex du pokemon
- Types du pokemon
- Taille en m du pokemon
- Poids en kg du pokemon
- Groupes d'oeufs du pokemon
- Couleur du pokemon
- Empreinte du pokemon
- Apparence du corps du pokemon
- Cri du pokemon en format mp3

In [5]:
#Looping on the pokemon urls to get the needed infos
with open("crawl_pokemon_data.csv","w",encoding ="utf-8") as file:
    print("url\tnom\tnumero\ttypes\ttaille\tpoids\tgroupe_oeufs\tcouleur\tempreinte\tcorps",file=file)

for i,url_fr in tqdm(enumerate(urls_fr)) :
    chrome.get(url_fr)
    time.sleep(0.5)
    
    #Name
    try:
        lecture_nom = chrome.find_element(By.CSS_SELECTOR,"h1[class='firstHeading mw-first-heading']")
        nom = lecture_nom.text
        nom=' '.join(nom.replace('\n','').split())
    except:
        nom=""
    
    #Numero pokedex
    try:
        lecture_numero_pokedex = chrome.find_element(By.XPATH,'//*[@id="mw-content-text"]/div[1]/table[2]/thead/tr/th[1]')
        numero_pokedex = lecture_numero_pokedex.text
        numero_pokedex=' '.join(numero_pokedex.replace('\n','').split())
    except:
        numero_pokedex=""
    
    #Table d'infos
    table_info = chrome.find_element(By.XPATH,'//*[@id="mw-content-text"]/div[1]/table[2]/tbody')
    rows = table_info.find_elements(By.TAG_NAME,"tr")
    #Types
    try:
        for row in rows :
            try :
                if row.find_element(By.TAG_NAME,"th").find_element(By.TAG_NAME,"a").get_attribute("title") =="Type" :
                    lecture_types = row.find_element(By.TAG_NAME,"td")
                    types =lecture_types.find_elements(By.TAG_NAME,"a")
                    types = [elem.get_attribute("title").replace(" (type)","")
                            for elem in types]
            except:
                pass
    except:
        types=[]
    
    #Taille
    try:
        for row in rows :
            try :
                if row.find_element(By.TAG_NAME,"th").find_element(By.TAG_NAME,"a").get_attribute("title") =="Taille" :
                    lecture_taille = row.find_element(By.TAG_NAME,"td")
                    taille =lecture_taille.text.split(" m,")[0].replace(",",".")
            except:
                pass
    except:
        taille=""
        
    #Poids
    try:
        for row in rows :
            try :
                if row.find_element(By.TAG_NAME,"th").find_element(By.TAG_NAME,"a").get_attribute("title") =="Poids" :
                    lecture_poids = row.find_element(By.TAG_NAME,"td")
                    poids =lecture_poids.text.split(" kg,")[0].replace(",",".")
            except:
                pass
    except:
        poids=""
    
    #Groupe oeufs
    try:
        for row in rows :
            try :
                if row.find_element(By.TAG_NAME,"th").find_element(By.TAG_NAME,"a").get_attribute("title") =="Liste des Pokémon par groupe d'Œuf" :
                    lecture_groupes = row.find_element(By.TAG_NAME,"td")
                    groupes =lecture_groupes.find_elements(By.TAG_NAME,"a")
                    groupes = [elem.get_attribute("title").replace("Catégorie:Pokémon du groupe ","")
                            for elem in groupes]
            except:
                pass
    except:
        groupes=[]
    
    #Couleur
    try:
        for row in rows :
            try :
                if row.find_element(By.TAG_NAME,"th").find_element(By.TAG_NAME,"a").get_attribute("title") =="Couleur" :
                    lecture_couleur = row.find_element(By.TAG_NAME,"td")
                    couleur =lecture_couleur.find_element(By.TAG_NAME,"img").get_attribute("alt")
            except:
                pass
    except:
        couleur=""
    
    #Empreinte
    try:
        for row in rows :
            try :
                if row.find_element(By.TAG_NAME,"th").text =="Empreinte" :
                    lecture_empreinte = row.find_element(By.TAG_NAME,"td")
                    empreinte =lecture_empreinte.find_element(By.TAG_NAME,"a").get_attribute("href")
                    empreinte = re.search("(\d+)",empreinte).group()
            except:
                pass
    except:
        empreinte=""
        
    #Apparence du corps
    try:
        for row in rows :
            try :
                if row.find_element(By.TAG_NAME,"th").find_element(By.TAG_NAME,"a").get_attribute("title") =="Apparence du corps" :
                    lecture_corps = row.find_element(By.TAG_NAME,"td")
                    corps =lecture_corps.find_element(By.TAG_NAME,"a").get_attribute("title")
                    corps = re.search("(\d+)",corps).group()
            except:
                pass
    except:
        corps=""
        
    #Audio cri
    chrome.get(urls_de[i])
    time.sleep(0.5)
    try : 
        lecture_audio = chrome.find_element(By.CSS_SELECTOR,"audio[class='mp3handler-audio']")
        audio_url = lecture_audio.find_element(By.TAG_NAME,'source').get_attribute("src")
        req = requests.get(audio_url)
        time.sleep(1)
        with open(f".\MP3\{nom}.mp3",'wb') as file :
            file.write(req.content)
    except Exception as e :
        with open("failed_audio.txt",'a',encoding ="utf-8") as file :
            print(nom+" === "+str(e),file=file)
    #Sauvegarder resultats
    try:
        result=(url_fr+'\t'+ str(nom)+ '\t'+ str(numero_pokedex)+ '\t'
                +str(types)+ '\t' +str(taille)+ '\t' +str(poids)+ '\t' 
                +str(groupes)+ '\t'+str(couleur)+ '\t'+str(empreinte)+ '\t' 
                +str(corps))
        with open('crawl_pokemon_data.csv','a',encoding='utf-8') as fhandle:
            print(result,file=fhandle)
        with open('completed.txt','a',encoding='utf-8') as fhandle:
                    print(url_fr,file=fhandle)
    except :
        exception=(url_fr+'\t'+ ""+ '\t'+ ""+ '\t'
                +"" '\t' +""+ '\t' +""+ '\t' 
                +""+ '\t'+""+ '\t'+""+ '\t'+"")
        with open('crawl_pokemon_data.csv','a',encoding='utf-8') as fhandle:
            print(exception,file=fhandle)

1234it [1:51:56,  5.44s/it]
